In [1]:
import numpy as np


## Linear Equations
$$
Ax = b
$$
### Solution 1:
- Fixed point iteration:

    define $G(x) = Ax - b + x = ( A + I)x - b$, and let $G(x)$ be the x in the next iteration
    
    The iterative process converges if all eigenvalues of (A+I) is within the unit circle. However, this condition is too restrict, which is unlikely to happen in real cases. Therefore, we use Gauss-Jacobi, Gauss-Seidel and other iterative shemes instead.
    
    

### Gauss-Jacobi algorithm

In [5]:
def gauss_jacobi(A,b, x):
    # Extract the diagonal entries
    N = np.diagflat(np.diag(A))
    P = N - A
    return np.linalg.solve(N, b + P.dot(x))

In [6]:

A = np.array([[3,1,2,-2],[1,5,2,-3],[2,3,6,3],[1,1,2,-5]])
x = np.zeros(shape = len(A))
b = np.random.uniform(high = 5,size=len(A))
epsilon = 1e-14
pre_x = np.zeros_like(x)
while True:
    pre_x = x
    x = gauss_jacobi(A,b,x)
    if  ((x - pre_x)**2).sum() < epsilon:
        print('Successfully find solutions:', x)
        break

Successfully find solutions: [-1.01708116  0.16642876  0.93157338 -0.72527202]


### Gauss-Seidel algorithm

In [7]:
def gauss_seidel(A,b, x,w = 1,shuffle = False):
        # Allowing dampening and extrapolating with weight w
        # The sequence of solving x's matters for Gauss-Seidel algorithm, we add a shuffle option
    if shuffle == True:
        values = np.concatenate([A,b.reshape(len(b),-1)],axis=1)
        A = values[:,:-1]
        b = values[:,-1]
    
    # Extract the diagonal entries, lower triangular, and upper triangular
    L = np.tril(A)
    U = A - L
        
    return w * np.linalg.solve(L, b - U.dot(x)) + (1 - w) * x

In [12]:
# Wrap the two linear system solves into a Linear Solver
class LinearSolver:
    
    # We allow direct call methods in the class
    @staticmethod
    def gauss_jacobi(A, b, tol=1e-10):
        x = np.zeros_like(b)
        N = np.diagflat(np.diag(A))
        P = A - N
        
        while True:
            pre_x = x.copy()
            x = np.linalg.solve(N, b - P.dot(pre_x))
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Jacobi method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x
    
    @staticmethod
    def gauss_seidel(A, b, w=1, shuffle=False, tol=1e-10):
        x = np.zeros_like(b)
        
        if shuffle:
            indices = np.arange(len(b))
            np.random.shuffle(indices)
            A = A[indices]
            b = b[indices]
        
        L = np.tril(A)
        U = A - L
        
        while True:
            pre_x = x.copy()
            x = w * np.linalg.solve(L, b - U.dot(pre_x)) + (1 - w) * pre_x
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Gauss-Seidel method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x


In [13]:
A = np.array([[1,.5,.3],[.6,1,.1],[.2,.4,1]])
b = np.array([5.,7.,4.])


In [14]:
lin_solver = LinearSolver()
lin_solver.gauss_jacobi(A,b)
lin_solver.gauss_seidel(A,b)

Successfully found solutions with Jacobi method: [1.67155639 5.86510467 1.31964982]
Results for Ax - b = [3.67428675e-06 3.48811842e-06 2.97013887e-06]
Successfully found solutions with Gauss-Seidel method: [1.67155553 5.86510185 1.31964815]
Results for Ax - b = [ 9.00075156e-07 -1.41752707e-08  0.00000000e+00]


array([1.67155553, 5.86510185, 1.31964815])

## Optimization

### Polytope
- Polytope method can be used for any $R^n$ functions, regardless of continuity property
- Polytope can guarantee a solution but cannot guarantee a global minimum (maximum)

In [389]:
def polytope_algorithm(f, simplex, epsilon):
    """
    Perform the Polytope Algorithm (also known as the Nelder-Mead method).
    
    Parameters:
    f (function): The objective function to minimize.
    simplex (numpy array): Initial simplex, an (n+1)x(n) array.
    epsilon (float): Stopping rule parameter.
    
    Returns:
    numpy array: The optimized vertex of the simplex.
    """
    
    n = simplex.shape[0]
    
    while True:
        # Step 1: Reorder the simplex vertices so that f(x^1) >= f(x^2) >= ... >= f(x^{n+1})
        simplex = np.array(sorted(simplex, key=lambda x: f(x), reverse=True))
        # Step 2: Find the least i such that f(x^i) > f(y^i) where y^i is the reflection of x^i
        for i in range(n):
            # We shrink the distance of the reflection
            reflected = 1e-3 * np.mean(np.delete(initial_simplex,i,0)) + simplex[i]
            # The first element that exceeds the function value of the reflected
            if f(simplex[i]) > f(reflected):
                simplex[i] = reflected
                break
        # Step 3: Stopping rule, when the simplex is small enough
        if np.sum(abs(simplex[-1] - simplex[0])) < epsilon:
            return simplex[-1]  # Return the best vertex
        # Step 4: Shrink simplex
        simplex = .5 * simplex[-1] + 0.5 * simplex

# Example usage:
def objective_function(x):
    return np.sum((x - np.array([1.3, 0.8,.3]))**2)  # Example: minimize the sum of squares






In [390]:
# For a 3-dimensional problem, simplex (the convex hull) has three vertex
initial_simplex = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])


result = polytope_algorithm(objective_function, initial_simplex, epsilon=1e-14)
print("Optimized vertex:", result)

Optimized vertex: [1.21677517 0.71625434 0.46697049]


### Newton Method

In [15]:
# For simplicity, I show a 2 dimensional case, and I write a BFGS optimizer more explicitly below 
x = np.array([1.0,5.0])
delta = 1e-10
# Newton's method
while 1:
    print('Current (x,y) = ', x[0], x[1])
    fx = pi(x)
    grads = grad_func(x)
    H = hessian(pi)(x)
    s = np.linalg.solve(H, -grads.reshape(-1,1)).flatten()
    
    if (s**2).sum() < epsilon * (1 + (x**2).sum()):
        print('Iteration ends')
        if (grads**2).sum() > delta  * (1 + np.abs(fx)):
            print('Not optimum')
        else:
            print('reach an optimum')
        print('Success')
        break
    x = x + s

Optimal solution: [10. 14.]


### Quasi-Newton method
#### BFGS

In [141]:
import autograd.numpy as np
from autograd import grad, hessian

class BFGS:
    """
    Description: This an example of a BFGS optimizer. 
    For faster convergence, I comment out the delta criterion
    """
    def __init__(self, func, x, epsilon=1e-12,delta = 1e-10):
        self.func = func
        self.x = np.array(x, dtype=float)
        self.epsilon = epsilon
        self.delta = 
        self.H = np.eye(len(x))
        self.grad_func = grad(func)
        self.hessian_func = hessian(func)
        
    def line_search(self, s, c1=0.6, c2=0.8):
        a = 0.5
        fx = self.func(self.x)
        grads = self.grad_func(self.x)
        
        while True:
            new_x = self.x + a * s
            new_grads = self.grad_func(new_x)
            new_fx = self.func(new_x)
            
            if (new_fx <= fx + c1 * a * (grads.dot(s)) and 
                np.abs(new_grads.dot(s)) < np.abs(c2 * (grads.dot(s)))):
                return a, new_x, new_grads,new_fx
            a *= 2
            if a > 4:
                return 4, new_x, new_grads,new_fx

    def update_hessian(self, z, y):
        H = self.H
        term1 = H - (H.dot(z.reshape(-1, 1)).dot(z.reshape(1, -1)).dot(H)) / (z.dot(H).dot(z))
        term2 = (y.reshape(-1, 1).dot(y.reshape(1, -1))) / (y.dot(z))
        return term1 + term2

    def optimize(self):
        while True:
            fx = self.func(self.x)
            grads = self.grad_func(self.x)
            H = self.hessian_func(self.x)
            s = np.linalg.solve(H, -grads.reshape(-1, 1)).flatten()
            
            lda, x_new, new_grads,new_fx = self.line_search(s)
            z = lda * s
            y = new_grads - grads
            print('Current x: ',self.x)
            self.x = x_new  # Update x to the new position calculated by line_search
            self.H = self.update_hessian(z, y)

            print('Current Function Value= ', fx)
            if (lda < 1e-10) or ((z**2).sum() < self.epsilon * (1 + (self.x**2).sum())):
                print('Iteration ends')
#                 if (grads**2).sum() > self.delta * (1 + np.abs(fx)):
#                     print('Not optimum')
#                 else:
#                     print('Reached an optimum')
                print('Success')
                break



SyntaxError: invalid syntax (1421773919.py, line 13)

In [159]:
# Simple Example
def fun(x):
    x0, x1,x2 = x
    u = x0**(.3) * x1**(.2)*x2**(.5)
    return (u-10)**2

bfgs = BFGS(func= fun, x=np.random.uniform(size=3))
bfgs.optimize()

## You can find a more useful example in the script ``logit_example'' in repo ``DiscreteChoiceModel_2024Summer'', 
## where I use BFGS optimizer to estimate the plain logit model using maximum likelihood estimator. 

Current x:  [0.60585548 0.89715616 0.68671143]
Current Function Value=  86.53270360660764
Current x:  [4.64470546 6.87792099 5.26457618]
Current Function Value=  21.633175901651885
Current x:  [6.66413045 9.8683034  7.55350856]
Current Function Value=  5.4082939754129695
Current x:  [ 7.67384294 11.36349461  8.69797474]
Current Function Value=  1.3520734938532455
Current x:  [ 8.17869919 12.11109021  9.27020784]
Current Function Value=  0.3380183734633103
Current x:  [ 8.43112732 12.48488802  9.55632439]
Current Function Value=  0.08450459336582758
Current x:  [ 8.55734138 12.67178692  9.69938266]
Current Function Value=  0.021126148341456895
Current x:  [ 8.62044841 12.76523637  9.7709118 ]
Current Function Value=  0.0052815370853640945
Current x:  [ 8.65200192 12.81196109  9.80667636]
Current Function Value=  0.0013203842713409592
Current x:  [ 8.66777868 12.83532346  9.82455865]
Current Function Value=  0.00033009606783527206
Current x:  [ 8.67566706 12.84700464  9.83349979]
Current